## Google Geocoding

* Now that everyone has an API Key (Congratulations!), it is time to start using it!

* You can utilize the Google Maps Geocoding API to turn addresses into latitudinal and longitudinal coordinates.

  * This process of converting an address to coordinates is called **geocoding**.

  * Since many APIs only understand locations formatted in terms of latitude/longitude, geocoding will be very valuable in translating addresses into data that APIs - like the Google Places API - can understand.

    * Google's API is not free and if credit card information is provided, Google will charge past a certain usage point. **It is a good idea to avoid pushing your API key to github by using adding the `config.py` to their `.gitignore` file or using environment variables.**

In [ ]:
# Dependencies
import requests
import json
import pandas as pd

from pprint import pprint

# Google developer API key
import sys
sys.path.append('../..')
from config import gkey
from config import owm_api


file = pd.read_csv("winedata.csv")

file.head()

In [ ]:
file["winery name"] = file["winery"] + " Winery"
file.head()

In [ ]:
just_brazil = file.loc[file["country"]=="Brazil"]

In [ ]:
just_brazil

In [ ]:
# Target location

winery_response =[]


for winery in just_brazil["winery name"]:

    target_location = winery
    
    # Build the endpoint URL
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target_location, gkey)
    
    # Run a request to endpoint and convert result to json
    geo_data = requests.get(target_url).json()
   
    winery_response.append(geo_data)        

In [ ]:
# Collect and store latitude and longitude for each winery
#winery_list = []
lat = []
lng = []

i=0

# Loop through wineries and append lists
for w in winery_response:
    try:
        latitude = winery_response[i]["results"][0]["geometry"]["location"]["lat"]
        longitude = winery_response[i]["results"][0]["geometry"]["location"]["lng"]
        
  #      winery_list.append(w)
        lat.append(latitude)
        lng.append(longitude)
        
        i = i+1
    except:
        lat.append("lat error")
        lng.append("lng error")
    
        i = i+1    

In [ ]:
# Add coordinates to dataframe
just_brazil["latitude"] = lat
just_brazil["longitude"] = lng

In [ ]:
# Drop rows without location data
#just_brazil = just_brazil.loc[just_brazil["latitude"] != "lat error"]

In [ ]:
import gmaps
import numpy as np

import time

# Configure gmaps with API key
gmaps.configure(api_key=gkey)


# Store 'Lat' and 'Lng' into  locations 
locations = just_brazil[["latitude", "longitude"]].astype(float)

In [90]:
# Create winery symbol layer
winery_layer = gmaps.marker_layer(
    locations
)


fig = gmaps.figure()
fig.add_layer(winery_layer)

fig

Figure(layout=FigureLayout(height='420px'))